# word2vec: How To Implement word2vec

### Explore Pre-trained Embeddings

Some other options:
- `glove-twitter-{25/50/100/200}`
- `glove-wiki-gigaword-{50/200/300}`
- `word2vec-google-news-300`
- `word2vec-ruscorpora-news-300`

In [1]:
# Install gensim
!pip install -U gensim

In [2]:
# Load pretrained word vectors using gensim
import gensim.downloader as api

wiki_embeddings = api.load('glove-wiki-gigaword-100')

In [3]:
# Explore the word vector for "king"
wiki_embeddings['king']

array([-0.32307 , -0.87616 ,  0.21977 ,  0.25268 ,  0.22976 ,  0.7388  ,
       -0.37954 , -0.35307 , -0.84369 , -1.1113  , -0.30266 ,  0.33178 ,
       -0.25113 ,  0.30448 , -0.077491, -0.89815 ,  0.092496, -1.1407  ,
       -0.58324 ,  0.66869 , -0.23122 , -0.95855 ,  0.28262 , -0.078848,
        0.75315 ,  0.26584 ,  0.3422  , -0.33949 ,  0.95608 ,  0.065641,
        0.45747 ,  0.39835 ,  0.57965 ,  0.39267 , -0.21851 ,  0.58795 ,
       -0.55999 ,  0.63368 , -0.043983, -0.68731 , -0.37841 ,  0.38026 ,
        0.61641 , -0.88269 , -0.12346 , -0.37928 , -0.38318 ,  0.23868 ,
        0.6685  , -0.43321 , -0.11065 ,  0.081723,  1.1569  ,  0.78958 ,
       -0.21223 , -2.3211  , -0.67806 ,  0.44561 ,  0.65707 ,  0.1045  ,
        0.46217 ,  0.19912 ,  0.25802 ,  0.057194,  0.53443 , -0.43133 ,
       -0.34311 ,  0.59789 , -0.58417 ,  0.068995,  0.23944 , -0.85181 ,
        0.30379 , -0.34177 , -0.25746 , -0.031101, -0.16285 ,  0.45169 ,
       -0.91627 ,  0.64521 ,  0.73281 , -0.22752 , 

In [4]:
# Find the words most similar to king based on the trained word vectors
wiki_embeddings.most_similar('king')

[('prince', 0.7682329416275024),
 ('queen', 0.7507690787315369),
 ('son', 0.7020888328552246),
 ('brother', 0.6985775828361511),
 ('monarch', 0.6977890729904175),
 ('throne', 0.691999077796936),
 ('kingdom', 0.6811409592628479),
 ('father', 0.6802029013633728),
 ('emperor', 0.6712858080863953),
 ('ii', 0.6676074266433716)]

### Train Our Own Model

In [5]:
# Read in the data and clean up column names
import gensim
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
pd.set_option('display.max_colwidth', 100)

messages = pd.read_csv('../../../data/spam.csv', encoding='latin-1')
messages = messages.drop(labels = ["Unnamed: 2", "Unnamed: 3", "Unnamed: 4"], axis = 1)
messages.columns = ["label", "text"]
messages.head()

,label,text
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there g..."
1,ham,Ok lar... Joking wif u oni...
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive ...
3,ham,U dun say so early hor... U c already then say...
4,ham,"Nah I don't think he goes to usf, he lives around here though"


In [6]:
# Clean data using the built in cleaner in gensim
messages['text_clean'] = messages['text'].apply(gensim.utils.simple_preprocess)
messages.head()

,label,text,text_clean
0,ham,"Go until jurong point, crazy.. Available only in bugis n great world la e buffet... Cine there g...","[go, until, jurong, point, crazy, available, only, in, bugis, great, world, la, buffet, cine, th..."
1,ham,Ok lar... Joking wif u oni...,"[ok, lar, joking, wif, oni]"
2,spam,Free entry in 2 a wkly comp to win FA Cup final tkts 21st May 2005. Text FA to 87121 to receive ...,"[free, entry, in, wkly, comp, to, win, fa, cup, final, tkts, st, may, text, fa, to, to, receive,..."
3,ham,U dun say so early hor... U c already then say...,"[dun, say, so, early, hor, already, then, say]"
4,ham,"Nah I don't think he goes to usf, he lives around here though","[nah, don, think, he, goes, to, usf, he, lives, around, here, though]"


In [7]:
# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(messages['text_clean'],
                                                    messages['label'], test_size=0.2)

In [8]:
X_train[:3]

3840                   [howz, pain, it, will, come, down, today, do, as, said, ystrday, ice, and, medicine]
4306                                                [guess, it, is, useless, calling, something, important]
3227    [six, chances, to, win, cash, from, to, pounds, txt, csh, and, send, to, cost, day, days, tsandc...
Name: text_clean, dtype: object

In [9]:
# Train the word2vec model
w2v_model = gensim.models.Word2Vec(X_train,
                                   vector_size=100,
                                   window=5,
                                   min_count=2)

In [10]:
# Explore the word vector for "king" base on our trained model
w2v_model.wv['king']

array([-0.19247977,  0.6190466 ,  0.04478936, -0.01983787,  0.00860529,
       -0.70188224,  0.4782202 ,  1.1245944 , -0.5012798 , -0.2595882 ,
       -0.06736855, -0.73101926,  0.12099461,  0.1591407 ,  0.17096047,
       -0.23417518,  0.31501403, -0.5395836 , -0.03882781, -1.0495489 ,
        0.29040635,  0.3205571 ,  0.38682568, -0.34803787, -0.12566078,
       -0.03773585, -0.623262  , -0.49068984, -0.44911402, -0.05638529,
        0.5208957 ,  0.03553074,  0.42916483, -0.56831753, -0.18553999,
        0.52741724,  0.14813967, -0.4269846 , -0.36978737, -0.838258  ,
        0.15760407, -0.47357145, -0.15807861,  0.00190849,  0.48561049,
       -0.10391472, -0.60782295, -0.09852116,  0.30184954,  0.5850612 ,
        0.30798447, -0.52475494, -0.17145552, -0.10280668, -0.10668403,
        0.2946952 ,  0.22838055,  0.00622786, -0.6389641 ,  0.14412853,
        0.15510799,  0.09088391, -0.0394561 , -0.05699762, -0.6537194 ,
        0.573346  ,  0.17329161,  0.5739335 , -0.7691256 ,  0.65

In [11]:
# Find the most similar words to "king" based on word vectors from our trained model
w2v_model.wv.most_similar('king')

[('my', 0.9996963739395142),
 ('all', 0.9996523857116699),
 ('the', 0.999615490436554),
 ('that', 0.9996038675308228),
 ('and', 0.9995888471603394),
 ('of', 0.9995670318603516),
 ('morning', 0.9995661973953247),
 ('make', 0.9995604157447815),
 ('well', 0.999557375907898),
 ('dear', 0.999554455280304)]